## LangChainとLangGraphによるRAG・AIエージェント[実践]入門
### 11.2　18のエージェントデザインパターン
- 1. パッシブゴールクリエイター（Passive Goal Creator）
#### passive_goal_creator

与えられた入力や状況から“暗黙の目標”を推定・生成するコンポーネント／設計パターン

入力クエリ => Prompt + LLM で目標を作成 => 次段へ

In [6]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from pydantic import BaseModel, Field


class Goal(BaseModel):
    description: str = Field(..., description="目標の説明")

    @property
    def text(self) -> str:
        return f"{self.description}"


class PassiveGoalCreator:
    def __init__(
        self,
        llm: ChatOpenAI,
    ):
        self.llm = llm

    def run(self, query: str) -> Goal:
        prompt = ChatPromptTemplate.from_template(
            "ユーザーの入力を分析し、明確で実行可能な目標を生成してください。\n"
            "要件:\n"
            "1. 目標は具体的かつ明確であり、実行可能なレベルで詳細化されている必要があります。\n"
            "2. あなたが実行可能な行動は以下の行動だけです。\n"
            "   - インターネットを利用して、目標を達成するための調査を行う。\n"
            "   - ユーザーのためのレポートを生成する。\n"
            "3. 決して2.以外の行動を取ってはいけません。\n"
            "ユーザーの入力: {query}"
        )
        chain = prompt | self.llm.with_structured_output(Goal) # JSON形式で出力
        return chain.invoke({"query": query})


def main():
    import argparse

    from settings import Settings

    settings = Settings()

    parser = argparse.ArgumentParser(
        description="PassiveGoalCreatorを利用して目標を生成します"
    )
    parser.add_argument("--task", type=str, help="実行するタスク", #required=True,
                        default="カレーライスの作り方")   # for Jupyter notebook
    args, _ = parser.parse_known_args()                 # for Jupyter notebook
    #args = parser.parse_args()
    print(f"## imput Task: {args.task}")

    llm = ChatOpenAI(
        model=settings.openai_smart_model, temperature=settings.temperature
    )
    goal_creator = PassiveGoalCreator(llm=llm)
    result: Goal = goal_creator.run(query=args.task)

    print(f"{result.text}")


if __name__ == "__main__":
    main()

### 目標: カレーライスの作り方を理解し、ユーザーにわかりやすく説明するレポートを作成する。

#### ステップ 1: インターネットを利用して調査を行う
- **具体的なレシピを検索**: カレーライスの基本的なレシピを探し、材料と手順を確認する。
- **調理のコツを調べる**: カレーライスを美味しく作るためのポイントやコツを調査する。
- **バリエーションを探す**: さまざまな種類のカレーライス（例: チキンカレー、ビーフカレー、ベジタリアンカレー）について情報を集める。

#### ステップ 2: ユーザーのためのレポートを生成する
- **レシピのまとめ**: 調査した情報をもとに、カレーライスの基本的なレシピをわかりやすくまとめる。
- **調理のコツの紹介**: 美味しく作るためのポイントをリストアップし、ユーザーに役立つ情報を提供する。
- **バリエーションの提案**: ユーザーが試してみたいと思うようなカレーライスのバリエーションを提案する。

#### 期待される成果
- ユーザーがカレーライスを作るための具体的な手順を理解し、実際に調理できるようになる。
- ユーザーがカレーライスのバリエーションを楽しむためのアイデアを得ることができる。

この目標を達成するために、インターネットを活用して必要な情報を収集し、ユーザーにとって有益なレポートを作成します。
